# Notebook description

In this notebook I've implemented an Authorship verification task using a the ononimus notebook pushlished on Kaggle in 2019. In addition to that implementation I've tried the proposed method on different datasets and also on dataset that wasn't in the training set. Moreover I've also added more metrics (tooken from PAN competitions) to have a better understanding on which are the datasets where the model performs better and why.

## (To just evaluate the datasets run the code after in the EVALUATION paragraph)

In [2]:
#Imports

import os
import re
import string
import pandas as pd
import numpy as np
import random
import ast  
import json


from time import time
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.callbacks import ModelCheckpoint
import csv
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import brier_score_loss
from sklearn.metrics import precision_score, recall_score, f1_score

Using TensorFlow backend.


# Implementation with Twitter-data

In [ ]:
#dataset loading from kaggle's inputs
dataset = pd.read_excel('/kaggle/input/twitter-data/train.xlsx')
dataset.head()

In [ ]:
index=range(0,1)
df=pd.DataFrame(index=index)

The dataset chosen consists in a balanced dataset containing 26k different tweets of 13 different authors, to be able to use it as a dataset for AV task it is necessary to firstly create different pairs of text some of wich will have the same author and some won't. In this case 200k pairs of tweets will be created and used for training, validation and testing.

In [ ]:
first = True
while (len(df)<200000):
    for user in dataset.user.unique():
        dataset = dataset.sample(frac=1).reset_index(drop=True)
        df1 = dataset[dataset['user'] == user][0:20].reset_index(drop=True)
        df2 = dataset[dataset['user'] == user][-10:].reset_index(drop=True)
        df3 = dataset[dataset['user'] != user][-10:].reset_index(drop=True)
        df3 = pd.concat([df2,df3],ignore_index=True)
        if first :
            df = pd.concat([df1,df3],axis = 1)
            first = False
        else:
            df = pd.concat([df,pd.concat([df1,df3],axis = 1)])

In [ ]:
df.columns = ['user1','text1','user2','text2']
df['is_same'] = 0
df.loc[df['user1']==df['user2'],'is_same'] = 1
df.loc[df['user1']!=df['user2'],'is_same'] = 0
df = df.sample(frac=1).reset_index(drop=True)

For this implementation the preprocessing steps are pretty simple, firstly the data is cleaned from special characters, then tokenized and enery word is stored in a dictionary and substituited in the dataset with a correspondig token.

In [ ]:
def text_to_word_list(tweet):
    ''' Pre process and convert texts to a list of words '''
    tweet = str(tweet)
    tweet = tweet.lower()
    tweet = tweet.lower()
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'[_"\-;%()|.,+&=*%]', '', tweet)
    tweet = re.sub(r'\.', ' . ', tweet)
    tweet = re.sub(r'\!', ' !', tweet)
    tweet = re.sub(r'\?', ' ?', tweet)
    tweet = re.sub(r'\,', ' ,', tweet)
    tweet = re.sub(r':', ' : ', tweet)
    tweet = re.sub(r'#', ' # ', tweet)
    tweet = re.sub(r'@', ' @ ', tweet)
    tweet = re.sub(r'd .c .', 'd.c.', tweet)
    tweet = re.sub(r'u .s .', 'd.c.', tweet)
    tweet = re.sub(r' amp ', ' and ', tweet)
    tweet = re.sub(r'pm', ' pm ', tweet)
    tweet = re.sub(r'news', ' news ', tweet)
    tweet = re.sub(r' . . . ', ' ', tweet)
    tweet = re.sub(r' .  .  . ', ' ', tweet)
    tweet = re.sub(r' ! ! ', ' ! ', tweet)
    tweet = re.sub(r'&amp', 'and', tweet)
    tweet = tweet.split()

    return tweet

vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
tweets_cols = ['text1', 'text2']
def preprocess(df):
    for index, row in df.iterrows():
        for tweet in tweets_cols:
                integers = []
                for word in text_to_word_list(row[tweet]):
                    if word not in vocabulary:
                        vocabulary[word] = len(inverse_vocabulary)
                        integers.append(len(inverse_vocabulary))
                        inverse_vocabulary.append(word)
                    else:
                        integers.append(vocabulary[word])

                df.set_value(index, tweet, integers)
    return df

In [ ]:
df=preprocess(df)
#after the preprocessing steps the max length is computed in order to then pad every text to the max length and have 
    #an omogeneus length in the samples.
max_length = max(df.text1.map(lambda x: len(x)).max(),
                     df.text2.map(lambda x: len(x)).max())
max_length

In [ ]:
df = df.sample(frac=1).reset_index(drop=True) #the data is shuffled before being splitted in train,validation and test

In [ ]:
#In the splitting process the data is divided first in 80% train and 20% test and then 10% of the 80% of the training is reserved for validation
X_train_tw, X_test_tw, Y_train_tw, Y_test_tw = train_test_split(df[tweets_cols], df['is_same'], test_size=0.2)
X_train_tw, X_validation_tw, Y_train_tw, Y_validation_tw = train_test_split(X_train_tw,Y_train_tw, test_size=0.1)

test_tw = df[df.index.isin(X_test_tw.index)]

X_train_tw = {'left': X_train_tw.text1, 'right': X_train_tw.text2}
X_validation_tw = {'left': X_validation_tw.text1, 'right': X_validation_tw.text2}
X_test_tw = {'left': X_test_tw.text1, 'right': X_test_tw.text2}

Y_train_tw = Y_train_tw.values
Y_validation_tw = Y_validation_tw.values
Y_test_tw = Y_test_tw.values

In [ ]:
#Padding all the train, test and validation set to the max length in the case of twitter data the max length of a tweet 
    # is 60 tokens
    
def making_pad(data):
    for key,val in data.items():
        data[key]=pad_sequences(data[key],maxlen=maxlength)
    return data

X_train_tw = making_pad(X_train_tw)
X_validation_tw = making_pad(X_validation_tw)
X_test_tw = making_pad(X_test_tw)

In [ ]:
assert X_train_tw['left'].shape == X_train_tw['right'].shape
assert len(X_train_tw['left']) == len(Y_train_tw)

X_train_tw['left'].shape

### Definition of the model

In [ ]:
# Import necessary modules from Keras
from keras import optimizers
from keras.layers import Input, Embedding, LSTM, Lambda, Dense
from keras.models import Model
import keras.backend as K

# Define some constants
n_hidden = 20  # Number of hidden units in the LSTM layer
batch_size = 1024  # Batch size for training
n_epoch = 10  # Number of epochs for training

# Function to calculate the exponent of the negative Manhattan distance between two vectors
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# Define input layers for left and right sequences
left_input = Input(shape=(max_length,), dtype='int32')
right_input = Input(shape=(max_length,), dtype='int32')

# Create an embedding layer for word embeddings
embedding_layer = Embedding(len(vocabulary)+1, 100,input_length=max_length)

# Apply the embedding layer to both input sequences
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Define a shared LSTM layer
shared_lstm = LSTM(n_hidden)

# Apply the shared LSTM to both encoded sequences
left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculate the distance between the outputs of the LSTM layers
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),
                         output_shape=lambda x: (x[0][0],1))([left_output, right_output])

# Create a Dense layer with sigmoid activation for prediction
prediction = Dense(1, activation='sigmoid')(malstm_distance)

# Define the model with inputs and outputs
model = Model([left_input, right_input], outputs=prediction)

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()


In [ ]:
# Start training
training_start_time = time()

history=model.fit([X_train_tw['left'], X_train_tw['right']], Y_train_tw, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation_tw['left'], X_validation_tw['right']], Y_validation_tw))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

In [ ]:
#first impression and possible evaluation of the model after training without counting the new metrics used for the complete evaluation 
preds = model.predict([X_test_tw['left'], X_test_tw['right']],batch_size=256)

preds[preds>=0.5]=1
preds[preds<0.5]=0

print(classification_report(Y_test_tw, preds))

Saving process of the trained model, and the test set in order to later evaluate the model

In [ ]:
model.save('modello_addestrato_su_tw')

my_dict = X_test_tw

serializable_dict = {}
for key, value in my_dict.items():
    if isinstance(value, np.ndarray):
        serializable_dict[key] = value.tolist()  # Convert NumPy array to list
    else:
        serializable_dict[key] = value

# Save dictionary as JSON
with open('X_test_TW.json', 'w') as f:
    json.dump(serializable_dict, f)
    
np.save('Y_test_tw.npy', Y_test_tw)

# Implementation with blog-authorship-corpus
In this case the dataset isn't more the one used in the original implementation but there had to be some small changes while working with the data. The data originally contained other labels other than the text and the it that contained information on the user that wrote the single blog post. For this method I removed them and was left with just id and text for every one of the more than 600k rows

In [ ]:
dataset = pd.read_csv('/kaggle/input/blog-authorship-corpus/blogtext.csv')
dataset = dataset.drop(columns=[col for col in dataset.columns if col not in ['id', 'text']])
dataset.head()

In [ ]:
print(len(dataset['id'].unique()))
index=range(0,1)
df=pd.DataFrame(index=index)

Also in this case I had to manually create the pairs needed to work in an AV task, so the code under does exactly this creating 1 million different pairs balanced with 500k pairs where the author is different and another 500k where the athor is the same

In [ ]:
def create_balanced_id_pairs(df, num_pairs=1000000):
    # Check if the DataFrame has enough data to form at least one pair
    if len(df) < 2:
        raise ValueError("The DataFrame must contain at least two rows to form a pair.")

    # Separate DataFrame by unique ids and filter out ids with less than two entries
    grouped = df.groupby('id').filter(lambda x: len(x) > 1).groupby('id')

    # List for storing temporary results
    pairs = []

    # Half the pairs with the same id
    same_id_pairs = num_pairs // 2
    different_id_pairs = num_pairs - same_id_pairs

    # Create pairs with the same id
    for name, group in grouped:
        if len(group) >= 2 and same_id_pairs > 0:
            for _ in range(min(same_id_pairs, len(group) // 2)):  # Limit to available pairs in group
                sampled_rows = group.sample(n=2)
                pairs.append({
                    'id1': sampled_rows.iloc[0]['id'],
                    'text1': sampled_rows.iloc[0]['text'],
                    'id2': sampled_rows.iloc[1]['id'],
                    'text2': sampled_rows.iloc[1]['text']
                })
                same_id_pairs -= 1

    # Create pairs with different ids
    all_ids = list(grouped.groups.keys())
    while different_id_pairs > 0 and len(all_ids) > 1:
        selected_ids = random.sample(all_ids, 2)
        group1, group2 = grouped.get_group(selected_ids[0]), grouped.get_group(selected_ids[1])
        row1, row2 = group1.sample(n=1).iloc[0], group2.sample(n=1).iloc[0]
        pairs.append({
            'id1': row1['id'],
            'text1': row1['text'],
            'id2': row2['id'],
            'text2': row2['text']
        })
        different_id_pairs -= 1
        

    # Create a new DataFrame from the pairs
    new_df = pd.DataFrame(pairs)
    
    return new_df

df = create_balanced_id_pairs(dataset)
print(df)

In [ ]:
df = df[['id1','text1','id2','text2']]
df['is_same'] = 0
df.loc[df['id1']==df['id2'],'is_same'] = 1
df.loc[df['id1']!=df['id2'],'is_same'] = 0
df = df.sample(frac=1).reset_index(drop=True)

In order to work more easily and computationally be able to train a new model with this method from scrath from the original 1 million pairs I removed all the ones that excedeed a certain arbitrary length of (in this case) 750. Doing this I'was left with a little more than 200k pairs, similar number as the number of different pairs used to train and test the model with the original implementation.

In [ ]:
indices_to_keep = []

# Iterate over each row using the index
for i in df.index:
    if len(df['text1'].loc[i]) <= 750:
        if len(df['text2'].loc[i]) <= 750:
            indices_to_keep.append(i)

# Use the list of indices to keep to filter the DataFrame
filtered_df = df.loc[indices_to_keep]
filtered.to_csv('filtered_df.csv', index=False) 
#The dataset is then saved to be reused later and more easily in a second moment when It'll be time to preprocess the data.

In [ ]:
def text_to_word_list(tweet):
    ''' Pre process and convert texts to a list of words '''
    tweet = str(tweet)
    tweet = tweet.lower()
    tweet = tweet.lower()
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'[_"\-;%()|.,+&=*%]', '', tweet)
    tweet = re.sub(r'\.', ' . ', tweet)
    tweet = re.sub(r'\!', ' !', tweet)
    tweet = re.sub(r'\?', ' ?', tweet)
    tweet = re.sub(r'\,', ' ,', tweet)
    tweet = re.sub(r':', ' : ', tweet)
    tweet = re.sub(r'#', ' # ', tweet)
    tweet = re.sub(r'@', ' @ ', tweet)
    tweet = re.sub(r'd .c .', 'd.c.', tweet)
    tweet = re.sub(r'u .s .', 'd.c.', tweet)
    tweet = re.sub(r' amp ', ' and ', tweet)
    tweet = re.sub(r'pm', ' pm ', tweet)
    tweet = re.sub(r'news', ' news ', tweet)
    tweet = re.sub(r' . . . ', ' ', tweet)
    tweet = re.sub(r' .  .  . ', ' ', tweet)
    tweet = re.sub(r' ! ! ', ' ! ', tweet)
    tweet = re.sub(r'&amp', 'and', tweet)
    tweet = tweet.split()

    return tweet

vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
tweets_cols = ['text1', 'text2']
def preprocess(df):
    for index, row in df.iterrows():
        for tweet in tweets_cols:
                integers = []
                for word in text_to_word_list(row[tweet]):
                    if word not in vocabulary:
                        vocabulary[word] = len(inverse_vocabulary)
                        integers.append(len(inverse_vocabulary))
                        inverse_vocabulary.append(word)
                    else:
                        integers.append(vocabulary[word])

                df.set_value(index, tweet, integers)
    return df

In [ ]:
#Loading the prevoiusly prepared and filtered dataset ready to be preprocessed
filtered_df = pd.read_csv('/kaggle/input/data-and-modelli/DATI PROGETTO AAA/filtered_df.csv')
filtered_df.head()

In [ ]:
df_preprocessed = preprocess(filtered_df) #the preprocessing steps are exactly the same as the original implementation,
#cleaning of the text, tokenization of the word and saving the words in a dictionary
max_length = max(df_preprocessed.text1.map(lambda x: len(x)).max(),
                     df_preprocessed.text2.map(lambda x: len(x)).max())
print(len(df_preprocessed), len(vocabulary),max_length) #In this case the max length will be higher than the implementation usgin the twitter data but not to much higher 
#and we'll be able to train another model on this dataset from scratch using the same architecture

In [ ]:
# shuffling and data splitting in train, validation and test, same as the original implementation above
df_preprocessed = df_preprocessed.sample(frac=1).reset_index(drop=True)

X_train, X_test, Y_train, Y_test = train_test_split(df_preprocessed[tweets_cols], df_preprocessed['is_same'], test_size=0.2)
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train,Y_train, test_size=0.1)

test = df_preprocessed[df_preprocessed.index.isin(X_test.index)]

X_train = {'left': X_train.text1, 'right': X_train.text2}
X_validation = {'left': X_validation.text1, 'right': X_validation.text2}
X_test = {'left': X_test.text1, 'right': X_test.text2}

Y_train = Y_train.values
Y_validation = Y_validation.values
Y_test = Y_test.values

In [ ]:
#padding to bring all the text to the same length, with the filtered dataet the max length will be 196 tokens for a single blog post
def making_pad(data):
    for key,val in data.items():
        data[key]=pad_sequences(data[key],maxlen=max_length)
    return data

X_train = making_pad(X_train)
X_validation = making_pad(X_validation)
X_test = making_pad(X_test)

In [ ]:
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

X_train['left'].shape

In [ ]:
# Import necessary modules from Keras
from keras import optimizers
from keras.layers import Input, Embedding, LSTM, Lambda, Dense
from keras.models import Model
import keras.backend as K

# Define some constants
n_hidden = 20  # Number of hidden units in the LSTM layer
batch_size = 1024  # Batch size for training
n_epoch = 10  # Number of epochs for training

# Function to calculate the exponent of the negative Manhattan distance between two vectors
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# Define input layers for left and right sequences
left_input = Input(shape=(max_length,), dtype='int32')
right_input = Input(shape=(max_length,), dtype='int32')

# Create an embedding layer for word embeddings
embedding_layer = Embedding(len(vocabulary)+1, 100,input_length=max_length)

# Apply the embedding layer to both input sequences
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Define a shared LSTM layer
shared_lstm = LSTM(n_hidden)

# Apply the shared LSTM to both encoded sequences
left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculate the distance between the outputs of the LSTM layers
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),
                         output_shape=lambda x: (x[0][0],1))([left_output, right_output])

# Create a Dense layer with sigmoid activation for prediction
prediction = Dense(1, activation='sigmoid')(malstm_distance)

# Define the model with inputs and outputs
model = Model([left_input, right_input], outputs=prediction)

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

In [ ]:
# Start training
training_start_time = time()

history=model.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

In [ ]:
#as in the previous case the model and the test set is saved in order to easilly evaluate later the method with this dataset
model.save('modello_addestrato_su_BAC')

my_dict = X_test

serializable_dict = {}
for key, value in my_dict.items():
    if isinstance(value, np.ndarray):
        serializable_dict[key] = value.tolist()  # Convert NumPy array to list
    else:
        serializable_dict[key] = value

# Save dictionary as JSON
with open('X_test_BAC.json', 'w') as f:
    json.dump(serializable_dict, f)

np.save('Y_test.npy', Y_test)

In [ ]:
preds = model.predict([X_test['left'], X_test['right']],batch_size=256)

preds[preds>=0.5]=1
preds[preds<0.5]=0

print(classification_report(Y_test, preds))

# TEST WITH PAN

In [ ]:
dataset = pd.read_json('/kaggle/input/pan20-av-train/pan20-authorship-verification-training-small/pan20-authorship-verification-training-small.jsonl', lines = True)
dataset.head()

In [ ]:
lunghezze = []
for i in range(len(dataset)):
    lunghezze.append(len(dataset['pair'].loc[i][0]))
    lunghezze.append(len(dataset['pair'].loc[i][1]))

In [ ]:
np.mean(lunghezze)

In [ ]:
def split_list(row):
    return row[0], row[1]

# Applica la funzione alla colonna combined_list e assegna i risultati a due nuove colonne
dataset[['text1', 'text2']] = dataset['pair'].apply(lambda x: pd.Series(split_list(x)))

# Rimuovi la colonna combined_list se necessario
dataset = dataset.drop(columns=['pair', 'fandoms'])

In [ ]:
dataset.head()

In [ ]:
truth = pd.read_json('/kaggle/input/pan20-av-train/pan20-authorship-verification-training-small/pan20-authorship-verification-training-small-truth.jsonl', lines = True)
truth.head()

In [ ]:
dataset['truth'] = truth['same'] 
dataset['truth'] = dataset['truth'].astype(int)
dataset.head()

In [ ]:
def text_to_word_list(tweet):
    ''' Pre process and convert texts to a list of words '''
    tweet = str(tweet)
    tweet = tweet.lower()
    tweet = tweet.lower()
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'[_"\-;%()|.,+&=*%]', '', tweet)
    tweet = re.sub(r'\.', ' . ', tweet)
    tweet = re.sub(r'\!', ' !', tweet)
    tweet = re.sub(r'\?', ' ?', tweet)
    tweet = re.sub(r'\,', ' ,', tweet)
    tweet = re.sub(r':', ' : ', tweet)
    tweet = re.sub(r'#', ' # ', tweet)
    tweet = re.sub(r'@', ' @ ', tweet)
    tweet = re.sub(r'd .c .', 'd.c.', tweet)
    tweet = re.sub(r'u .s .', 'd.c.', tweet)
    tweet = re.sub(r' amp ', ' and ', tweet)
    tweet = re.sub(r'pm', ' pm ', tweet)
    tweet = re.sub(r'news', ' news ', tweet)
    tweet = re.sub(r' . . . ', ' ', tweet)
    tweet = re.sub(r' .  .  . ', ' ', tweet)
    tweet = re.sub(r' ! ! ', ' ! ', tweet)
    tweet = re.sub(r'&amp', 'and', tweet)
    tweet = tweet.split()

    return tweet

vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
tweets_cols = ['text1', 'text2']
def preprocess(df):
    for index, row in df.iterrows():
        for tweet in tweets_cols:
                integers = []
                for word in text_to_word_list(row[tweet]):
                    if word not in vocabulary:
                        vocabulary[word] = len(inverse_vocabulary)
                        integers.append(len(inverse_vocabulary))
                        inverse_vocabulary.append(word)
                    else:
                        integers.append(vocabulary[word])

                df.set_value(index, tweet, integers)
    return df

In [ ]:
PAN_preprocessed = preprocess(dataset)

In [ ]:
PAN_preprocessed.head()

In [ ]:
len(PAN_preprocessed)

In [ ]:
indices_to_keep = []

# Iterate over each row using the index
for i in PAN_preprocessed.index:
    if len(PAN_preprocessed['text1'].loc[i]) <= 25000:
        if len(PAN_preprocessed['text2'].loc[i]) <= 25000:
            indices_to_keep.append(i)

# Use the list of indices to keep to filter the DataFrame
PAN_preprocessed = PAN_preprocessed.loc[indices_to_keep]
len(PAN_preprocessed)

In [ ]:
PAN_preprocessed = PAN_preprocessed.sample(frac=1).reset_index(drop=True)

X_train_PAN, X_test_PAN, Y_train_PAN, Y_test_PAN = train_test_split(PAN_preprocessed[tweets_cols], PAN_preprocessed['truth'], test_size=0.2)
X_train_PAN, X_validation_PAN, Y_train_PAN, Y_validation_PAN = train_test_split(X_train_PAN,Y_train_PAN, test_size=0.1)

test = PAN_preprocessed[PAN_preprocessed.index.isin(X_test_PAN.index)]

X_train_PAN = {'left': X_train_PAN.text1, 'right': X_train_PAN.text2}
X_validation_PAN = {'left': X_validation_PAN.text1, 'right': X_validation_PAN.text2}
X_test_PAN = {'left': X_test_PAN.text1, 'right': X_test_PAN.text2}

Y_train_PAN = Y_train_PAN.values
Y_validation_PAN = Y_validation_PAN.values
Y_test_PAN = Y_test_PAN.values

In [ ]:
max_length = max(PAN_preprocessed.text1.map(lambda x: len(x)).max(),
                     PAN_preprocessed.text2.map(lambda x: len(x)).max())
max_length

In [ ]:
def making_pad(data):
    for key,val in data.items():
        data[key]=pad_sequences(data[key],maxlen=max_length)
    return data

X_train_PAN = making_pad(X_train_PAN)
X_validation_PAN = making_pad(X_validation_PAN)
X_test_PAN = making_pad(X_test_PAN)

In [ ]:
assert X_train_PAN['left'].shape == X_train_PAN['right'].shape
assert len(X_train_PAN['left']) == len(Y_train_PAN)

X_train_PAN['left'].shape

In [ ]:
my_dict = X_test_PAN

serializable_dict = {}
for key, value in my_dict.items():
    if isinstance(value, np.ndarray):
        serializable_dict[key] = value.tolist()  # Convert NumPy array to list
    else:
        serializable_dict[key] = value

# Save dictionary as JSON
with open('X_test_PAN.json', 'w') as f:
    json.dump(serializable_dict, f)
    
np.save('Y_test_PAN.npy', Y_test_PAN)

Due to the extremely large number of tokens in each text in this dataset, training a model from scratch on this data proved to be impossible. In order to address this challenge, I trained another model named "modello_addestrato_su_BAC_senza_limiti_di_lunghezza". This model essentially replicates the same architecture as the original model but was trained on a filtered version of the Blog Authorship Corpus dataset. Importantly, in this model, there is no predetermined limit on the length of the tokens provided as input, enabling it to handle longer text pairs.

The code provided below, which involves creating and training the model with the PAN training set, serves as an example and illustration of the process. However, it's worth noting that the free GPU P100 available on Kaggle Notebooks was insufficient for training a model of this scale.

In [ ]:
from keras import optimizers
from keras.layers import merge ,Dense,Concatenate
n_hidden = 20
batch_size = 32
n_epoch = 10
max_length = None


def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


left_input = Input(shape=(max_length,), dtype='int32')
right_input = Input(shape=(max_length,), dtype='int32')

embedding_layer = Embedding((len(vocabulary)//2)+1, 100 ,input_length=max_length)

encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

shared_lstm = LSTM(n_hidden)
left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)
# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0],1))([left_output,right_output])
prediction = Dense(1,activation='sigmoid')(malstm_distance)
# Pack it all up into a model
model = Model([left_input, right_input],outputs=prediction)

# Adadelta optimizer, with gradient clipping by norm
# optimizer = optimizers.Adam(clipnorm=gradient_clipping_norm)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
# Start training
training_start_time = time()

history=model.fit([X_train_PAN['left'], X_train_PAN['right']], Y_train_PAN, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation_PAN['left'], X_validation_PAN['right']], Y_validation_PAN))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

# TEST WITH PAN22

In [ ]:
dataset = pd.read_json('/kaggle/input/data-and-modelli/pan22-authorship-verification-training.jsonl', lines = True)
dataset.head()

In [ ]:
lunghezze = []
for i in range(len(dataset)):
    lunghezze.append(len(dataset['pair'].loc[i][0]))
    lunghezze.append(len(dataset['pair'].loc[i][1]))
np.mean(lunghezze)

In [ ]:
def split_list(row):
    return row[0], row[1]

# Applica la funzione alla colonna combined_list e assegna i risultati a due nuove colonne
dataset[['text1', 'text2']] = dataset['pair'].apply(lambda x: pd.Series(split_list(x)))

# Rimuovi la colonna combined_list se necessario
dataset = dataset.drop(columns=['pair', 'discourse_types'])
dataset.head()

In [ ]:
truth = pd.read_json('/kaggle/input/data-and-modelli/pan22-authorship-verification-training-truth.jsonl', lines = True)
dataset['truth'] = truth['same'] 
dataset['truth'] = dataset['truth'].astype(int)
dataset.head()

In [ ]:
def text_to_word_list(tweet):
    ''' Pre process and convert texts to a list of words '''
    tweet = str(tweet)
    tweet = tweet.lower()
    tweet = tweet.lower()
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'[_"\-;%()|.,+&=*%]', '', tweet)
    tweet = re.sub(r'\.', ' . ', tweet)
    tweet = re.sub(r'\!', ' !', tweet)
    tweet = re.sub(r'\?', ' ?', tweet)
    tweet = re.sub(r'\,', ' ,', tweet)
    tweet = re.sub(r':', ' : ', tweet)
    tweet = re.sub(r'#', ' # ', tweet)
    tweet = re.sub(r'@', ' @ ', tweet)
    tweet = re.sub(r'd .c .', 'd.c.', tweet)
    tweet = re.sub(r'u .s .', 'd.c.', tweet)
    tweet = re.sub(r' amp ', ' and ', tweet)
    tweet = re.sub(r'pm', ' pm ', tweet)
    tweet = re.sub(r'news', ' news ', tweet)
    tweet = re.sub(r' . . . ', ' ', tweet)
    tweet = re.sub(r' .  .  . ', ' ', tweet)
    tweet = re.sub(r' ! ! ', ' ! ', tweet)
    tweet = re.sub(r'&amp', 'and', tweet)
    tweet = tweet.split()

    return tweet

vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
tweets_cols = ['text1', 'text2']
def preprocess(df):
    for index, row in df.iterrows():
        for tweet in tweets_cols:
                integers = []
                for word in text_to_word_list(row[tweet]):
                    if word not in vocabulary:
                        vocabulary[word] = len(inverse_vocabulary)
                        integers.append(len(inverse_vocabulary))
                        inverse_vocabulary.append(word)
                    else:
                        integers.append(vocabulary[word])

                df.set_value(index, tweet, integers)
    return df

In [ ]:
PAN22_preprocessed = preprocess(dataset)
PAN22_preprocessed.head()

In [ ]:
max_length = max(PAN22_preprocessed.text1.map(lambda x: len(x)).max(),
                     PAN22_preprocessed.text2.map(lambda x: len(x)).max())
max_length

In [ ]:
PAN22_preprocessed = PAN22_preprocessed.sample(frac=1).reset_index(drop=True)

X_train_PAN22, X_test_PAN22, Y_train_PAN22, Y_test_PAN22 = train_test_split(PAN22_preprocessed[tweets_cols], PAN22_preprocessed['truth'], test_size=0.2)
X_train_PAN22, X_validation_PAN22, Y_train_PAN22, Y_validation_PAN22 = train_test_split(X_train_PAN22,Y_train_PAN22, test_size=0.1)

test = PAN22_preprocessed[PAN22_preprocessed.index.isin(X_test_PAN22.index)]

X_train_PAN22 = {'left': X_train_PAN22.text1, 'right': X_train_PAN22.text2}
X_validation_PAN22 = {'left': X_validation_PAN22.text1, 'right': X_validation_PAN22.text2}
X_test_PAN22 = {'left': X_test_PAN22.text1, 'right': X_test_PAN22.text2}

Y_train_PAN22 = Y_train_PAN22.values
Y_validation_PAN22 = Y_validation_PAN22.values
Y_test_PAN22 = Y_test_PAN22.values

In [ ]:
def making_pad(data):
    for key,val in data.items():
        data[key]=pad_sequences(data[key],maxlen=max_length)
    return data

X_train_PAN22 = making_pad(X_train_PAN22)
X_validation_PAN22 = making_pad(X_validation_PAN22)
X_test_PAN22 = making_pad(X_test_PAN22)

In [ ]:
assert X_train_PAN22['left'].shape == X_train_PAN22['right'].shape
assert len(X_train_PAN22['left']) == len(Y_train_PAN22)

X_train_PAN22['left'].shape

In [ ]:
# Import necessary modules from Keras
from keras import optimizers
from keras.layers import Input, Embedding, LSTM, Lambda, Dense
from keras.models import Model
import keras.backend as K

# Define some constants
n_hidden = 20  # Number of hidden units in the LSTM layer
batch_size = 256  # Batch size for training
n_epoch = 10  # Number of epochs for training

# Function to calculate the exponent of the negative Manhattan distance between two vectors
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# Define input layers for left and right sequences
left_input = Input(shape=(max_length,), dtype='int32')
right_input = Input(shape=(max_length,), dtype='int32')

# Create an embedding layer for word embeddings
embedding_layer = Embedding(len(vocabulary)+1, 100,input_length=max_length)

# Apply the embedding layer to both input sequences
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Define a shared LSTM layer
shared_lstm = LSTM(n_hidden)

# Apply the shared LSTM to both encoded sequences
left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculate the distance between the outputs of the LSTM layers
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),
                         output_shape=lambda x: (x[0][0],1))([left_output, right_output])

# Create a Dense layer with sigmoid activation for prediction
prediction = Dense(1, activation='sigmoid')(malstm_distance)

# Define the model with inputs and outputs
model = Model([left_input, right_input], outputs=prediction)

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

In [ ]:
# Start training
training_start_time = time()

history=model.fit([X_train_PAN22['left'], X_train_PAN22['right']], Y_train_PAN22, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation_PAN22['left'], X_validation_PAN22['right']], Y_validation_PAN22))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

In [ ]:
model.save('modello_addestrato_su_PAN22')

my_dict = X_test_PAN22

serializable_dict = {}
for key, value in my_dict.items():
    if isinstance(value, np.ndarray):
        serializable_dict[key] = value.tolist()  # Convert NumPy array to list
    else:
        serializable_dict[key] = value

# Save dictionary as JSON
with open('X_test_PAN22.json', 'w') as f:
    json.dump(serializable_dict, f)
    
np.save('Y_test_PAN22.npy', Y_test_PAN22)

# EVALUATION

The evaluation metrics for these models were selected based on previous PAN Author Verification competitions and commonly used metrics such as precision and recall. The code to calculate C@1 and AUC was directly taken from the PAN competition's official repository. Some metrics were employed to reward models that produced a high number of uncertain responses rather than simply favoring a higher number of correct responses. To calculate these metrics, a custom threshold was utilized to determine when a prediction was uncertain (typically between 0.45 and 0.55), and such predictions were labeled as abstained predictions by the model.

In [4]:
def c_at_1(true_y, pred_y, threshold=5):
    """
    Calculates the c@1 score, an evaluation method specific to the
    PAN competition. This method rewards predictions which leave
    some problems unanswered (score = 0.5). See:

        A. Peñas and A. Rodrigo. A Simple Measure to Assess Nonresponse.
        In Proc. of the 49th Annual Meeting of the Association for
        Computational Linguistics, Vol. 1, pages 1415-1424, 2011.

    Parameters
    ----------
    prediction_scores : array [n_problems]

        The predictions outputted by a verification system.
        Assumes `0 >= prediction <=1`.

    ground_truth_scores : array [n_problems]

        The gold annotations provided for each problem.
        Will always be `0` or `1`.

    Returns
    ----------
    c@1 = the c@1 measure (which accounts for unanswered
        problems.)


    References
    ----------
        - E. Stamatatos, et al. Overview of the Author Identification
        Task at PAN 2014. CLEF (Working Notes) 2014: 877-897.
        - A. Peñas and A. Rodrigo. A Simple Measure to Assess Nonresponse.
        In Proc. of the 49th Annual Meeting of the Association for
        Computational Linguistics, Vol. 1, pages 1415-1424, 2011.

    """

    n = float(len(pred_y))
    nc, nu = 0.0, 0.0

    for gt_score, pred_score in zip(true_y, pred_y):
        if pred_score == threshold:
            nu += 1
        elif (pred_score > 0.5) == (gt_score > 0.5):
            nc += 1.0
    
    return (1 / n) * (nc + (nu * nc / n))


def auc(true_y, pred_y):
    """
    Calculates the AUC score (Area Under the Curve), a well-known
    scalar evaluation score for binary classifiers. This score
    also considers "unanswered" problem, where score = 0.5.

    Parameters
    ----------
    prediction_scores : array [n_problems]

        The predictions outputted by a verification system.
        Assumes `0 >= prediction <=1`.

    ground_truth_scores : array [n_problems]

        The gold annotations provided for each problem.
        Will typically be `0` or `1`.

    Returns
    ----------
    auc = the Area Under the Curve.

    References
    ----------
        E. Stamatatos, et al. Overview of the Author Identification
        Task at PAN 2014. CLEF (Working Notes) 2014: 877-897.

    """
    try:
        return roc_auc_score(true_y, pred_y)
    except ValueError:
        return 0.0

In [5]:
#loading back all the models and all the test set for each dataset

import keras
from keras import backend as K
from keras.models import load_model

# Define the exponent_neg_manhattan_distance function
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left - right), axis=1, keepdims=True))

# Register custom function with Keras
keras.utils.get_custom_objects()['exponent_neg_manhattan_distance'] = exponent_neg_manhattan_distance

# Clear TensorFlow session
K.clear_session()

# Load the model
model_BAC = load_model('/kaggle/input/data-and-modelli/DATI PROGETTO AAA/modello_addestrato_su_BAC (1)')
model_TW = load_model('/kaggle/input/data-and-modelli/DATI PROGETTO AAA/modello_addestrato_su_tw')
model_no_limits_BAC = load_model('/kaggle/input/data-and-modelli/Modello_addestrato_su_BAC_senza_limit_lunghezza')
model_trained_on_PAN22 = load_model('/kaggle/input/data-and-modelli/modello_addestrato_su_PAN22')

with open('/kaggle/input/data-and-modelli/DATI PROGETTO AAA/X_test_BAC.json', 'r') as f:
    test_BAC = json.load(f)
Y_test_BAC = np.load('/kaggle/input/data-and-modelli/DATI PROGETTO AAA/Y_test.npy')

with open('/kaggle/input/data-and-modelli/DATI PROGETTO AAA/X_test_TW.json', 'r') as f:
    test_TW = json.load(f)
Y_test_TW = np.load('/kaggle/input/data-and-modelli/DATI PROGETTO AAA/Y_test_tw.npy')

with open('/kaggle/input/data-and-modelli/X_test_PAN.json', 'r') as f:
    test_PAN = json.load(f)
Y_test_PAN = np.load('/kaggle/input/data-and-modelli/Y_test_PAN.npy')

with open('/kaggle/input/data-and-modelli/X_test_PAN22.json', 'r') as f:
    test_PAN22 = json.load(f)
Y_test_PAN22 = np.load('/kaggle/input/data-and-modelli/Y_test_PAN22.npy')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 258648500 elements. This may consume a large amount of memory.
  num_elements)
/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 258648500 elements. This may consume a large amount of memory.
  num_elements)


For the blog authorship corpus there are two different evaluation, one using the model trained on fixed length for the input and one with the model also used to evaluate the PAN test set trained without a predefined length for the input

In [6]:
preds_BAC = model_BAC.predict([test_BAC['left'], test_BAC['right']],batch_size=256)

preds_BAC_probabilities = preds_BAC

preds_BAC_no_astenuti = preds_BAC
preds_BAC_no_astenuti[preds_BAC>=0.5]=1
preds_BAC_no_astenuti[preds_BAC<0.5]=0

preds_BAC[preds_BAC>=0.55]=1
preds_BAC[preds_BAC<0.45]=0
preds_BAC[(preds_BAC >= 0.45) & (preds_BAC < 0.55)] = 5

#********************************************************************************************************************************************************************************

precision, recall, f_beta, _ = precision_recall_fscore_support(Y_test_BAC, preds_BAC_no_astenuti, beta=0.5, average="binary")
brier_score = 1-brier_score_loss(Y_test_BAC, preds_BAC_probabilities)
roc_auc = roc_auc_score(Y_test_BAC, preds_BAC)
c_at_1_score = c_at_1(Y_test_BAC, preds_BAC)
precision = precision_score(Y_test_BAC, preds_BAC_no_astenuti)
recall = recall_score(Y_test_BAC, preds_BAC_no_astenuti)
f1 = f1_score(Y_test_BAC, preds_BAC_no_astenuti)

results_df = pd.DataFrame({
    'Metric': ['ROC AUC Score', 'c_at_1', 'F0.5', 'Brier Score', 'Precision', 'Recall', 'f1-score'],
    'Value': [roc_auc, c_at_1_score, f_beta, brier_score, precision, recall, f1]
})

print(results_df)

          Metric     Value
0  ROC AUC Score  0.728207
1         c_at_1  0.734298
2           F0.5  0.770077
3    Brier Score  0.734298
4      Precision  0.770628
5         Recall  0.767882
6       f1-score  0.769253


In [7]:
preds_BAC = model_no_limits_BAC.predict([test_BAC['left'], test_BAC['right']],batch_size=256)

preds_BAC_probabilities = preds_BAC

preds_BAC_no_astenuti = preds_BAC
preds_BAC_no_astenuti[preds_BAC>=0.5]=1
preds_BAC_no_astenuti[preds_BAC<0.5]=0

preds_BAC[preds_BAC>=0.55]=1
preds_BAC[preds_BAC<0.45]=0
preds_BAC[(preds_BAC >= 0.45) & (preds_BAC < 0.55)] = 5

#********************************************************************************************************************************************************************************

precision, recall, f_beta, _ = precision_recall_fscore_support(Y_test_BAC, preds_BAC_no_astenuti, beta=0.5, average="binary")
brier_score = 1-brier_score_loss(Y_test_BAC, preds_BAC_probabilities)
roc_auc = roc_auc_score(Y_test_BAC, preds_BAC)
c_at_1_score = c_at_1(Y_test_BAC, preds_BAC)
precision = precision_score(Y_test_BAC, preds_BAC_no_astenuti)
recall = recall_score(Y_test_BAC, preds_BAC_no_astenuti)
f1 = f1_score(Y_test_BAC, preds_BAC_no_astenuti)

results_df = pd.DataFrame({
    'Metric': ['ROC AUC Score', 'c_at_1', 'F0.5', 'Brier Score', 'Precision', 'Recall', 'f1-score'],
    'Value': [roc_auc, c_at_1_score, f_beta, brier_score, precision, recall, f1]
})

print(results_df)

          Metric     Value
0  ROC AUC Score  0.854158
1         c_at_1  0.855872
2           F0.5  0.879012
3    Brier Score  0.855872
4      Precision  0.882504
5         Recall  0.865320
6       f1-score  0.873827


In [8]:
preds_TW = model_TW.predict([test_TW['left'], test_TW['right']],batch_size=256)

preds_TW_probabilities = preds_TW

preds_TW_no_astenuti = preds_TW
preds_TW_no_astenuti[preds_TW>=0.5]=1
preds_TW_no_astenuti[preds_TW<0.5]=0

preds_TW[preds_TW>=0.55]=1
preds_TW[preds_TW<0.45]=0
preds_TW[(preds_TW >= 0.45) & (preds_TW < 0.55)] = 5

#*******************************************************************************************************************************************************************************************************************************

precision, recall, f_beta, _ = precision_recall_fscore_support(Y_test_TW, preds_TW_no_astenuti, beta=0.5, average="binary")
brier_score = 1-brier_score_loss(Y_test_TW, preds_TW_probabilities)
roc_auc = roc_auc_score(Y_test_TW, preds_TW)
c_at_1_score = c_at_1(Y_test_TW, preds_TW)
precision = precision_score(Y_test_TW, preds_TW_no_astenuti)
recall = recall_score(Y_test_TW, preds_TW_no_astenuti)
f1 = f1_score(Y_test_TW, preds_TW_no_astenuti)

results_df = pd.DataFrame({
    'Metric': ['ROC AUC Score', 'c_at_1', 'F0.5', 'Brier Score', 'Precision', 'Recall', 'f1-score'],
    'Value': [roc_auc, c_at_1_score, f_beta, brier_score, precision, recall, f1]
})

print(results_df)

          Metric     Value
0  ROC AUC Score  0.877343
1         c_at_1  0.877348
2           F0.5  0.889363
3    Brier Score  0.877348
4      Precision  0.900095
5         Recall  0.848878
6       f1-score  0.873737


In [9]:
preds_PAN = model_no_limits_BAC.predict([test_PAN['left'], test_PAN['right']],batch_size=256)

preds_PAN_probabilities = preds_PAN

preds_PAN_no_astenuti = preds_PAN
preds_PAN_no_astenuti[preds_PAN>=0.5]=1
preds_PAN_no_astenuti[preds_PAN<0.5]=0

preds_PAN[preds_PAN>=0.55]=1
preds_PAN[preds_PAN<0.45]=0
preds_PAN[(preds_PAN >= 0.45) & (preds_PAN < 0.55)] = 5

#***********************************************************************************************************************************************************

precision, recall, f_beta, _ = precision_recall_fscore_support(Y_test_PAN, preds_PAN_no_astenuti, beta=0.5, average="binary")
brier_score = 1-brier_score_loss(Y_test_PAN, preds_PAN_probabilities)
roc_auc = roc_auc_score(Y_test_PAN, preds_PAN)
c_at_1_score = c_at_1(Y_test_PAN, preds_PAN)
precision = precision_score(Y_test_PAN, preds_PAN_no_astenuti)
recall = recall_score(Y_test_PAN, preds_PAN_no_astenuti)
f1 = f1_score(Y_test_PAN, preds_PAN_no_astenuti)

results_df = pd.DataFrame({
    'Metric': ['ROC AUC Score', 'c_at_1', 'F0.5', 'Brier Score', 'Precision', 'Recall', 'f1-score'],
    'Value': [roc_auc, c_at_1_score, f_beta, brier_score, precision, recall, f1]
})

print(results_df)

          Metric     Value
0  ROC AUC Score  0.502423
1         c_at_1  0.481175
2           F0.5  0.383312
3    Brier Score  0.481175
4      Precision  0.539588
5         Recall  0.177583
6       f1-score  0.267222


In [10]:
preds_PAN22 = model_trained_on_PAN22.predict([test_PAN22['left'], test_PAN22['right']],batch_size=256)

preds_PAN22_probabilities = preds_PAN22

preds_PAN22_no_astenuti = preds_PAN22
preds_PAN22_no_astenuti[preds_PAN22>=0.5]=1
preds_PAN22_no_astenuti[preds_PAN22<0.5]=0

preds_PAN22[preds_PAN22>=0.55]=1
preds_PAN22[preds_PAN22<0.45]=0
preds_PAN22[(preds_PAN22 >= 0.45) & (preds_PAN22 < 0.55)] = 5

#***********************************************************************************************************************************************************

precision, recall, f_beta, _ = precision_recall_fscore_support(Y_test_PAN22, preds_PAN22_no_astenuti, beta=0.5, average="binary")
brier_score = 1-brier_score_loss(Y_test_PAN22, preds_PAN22_probabilities)
roc_auc = roc_auc_score(Y_test_PAN22, preds_PAN22)
c_at_1_score = c_at_1(Y_test_PAN22, preds_PAN22)
precision = precision_score(Y_test_PAN22, preds_PAN22_no_astenuti)
recall = recall_score(Y_test_PAN22, preds_PAN22_no_astenuti)
f1 = f1_score(Y_test_PAN22, preds_PAN22_no_astenuti)

results_df = pd.DataFrame({
    'Metric': ['ROC AUC Score', 'c_at_1', 'F0.5', 'Brier Score', 'Precision', 'Recall', 'f1-score'],
    'Value': [roc_auc, c_at_1_score, f_beta, brier_score, precision, recall, f1]
})

print(results_df)

          Metric     Value
0  ROC AUC Score  0.756057
1         c_at_1  0.753771
2           F0.5  0.779221
3    Brier Score  0.753771
4      Precision  0.802893
5         Recall  0.697017
6       f1-score  0.746218
